In [1]:
import pandas as pd

In [2]:
pd.read_csv('accuracy.csv', parse_dates=True).groupby(['model', 'informed', 'use_ED_visits'])['WIS'].mean()

model                informed  use_ED_visits
oneStrain            False     False            0.107723
                               True             0.099522
                     True      False            0.083015
                               True             0.085229
sequentialTwoStrain  False     False            0.108011
                               True             0.103508
                     True      False            0.084234
                               True             0.077877
Name: WIS, dtype: float64

In [3]:
pd.read_csv('accuracy.csv', parse_dates=True).groupby(['model','informed', 'use_ED_visits', 'season'])['WIS'].mean()

model                informed  use_ED_visits  season   
oneStrain            False     False          2014-2015    0.134766
                                              2015-2016    0.129770
                                              2016-2017    0.123056
                                              2017-2018    0.094468
                                              2018-2019    0.095871
                                              2019-2020    0.112667
                                              2023-2024    0.067112
                               True           2014-2015    0.120413
                                              2015-2016    0.111644
                                              2016-2017    0.125290
                                              2017-2018    0.093212
                                              2018-2019    0.087227
                                              2019-2020    0.109994
                                              2023-2024    0